<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Info" data-toc-modified-id="Info-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Info</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset</a></span></li><li><span><a href="#Preliminary" data-toc-modified-id="Preliminary-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preliminary</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#All-data-missing-per-row-distribution" data-toc-modified-id="All-data-missing-per-row-distribution-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>All data missing per row distribution</a></span></li><li><span><a href="#Number-of-available-variables-per-row" data-toc-modified-id="Number-of-available-variables-per-row-3.0.2"><span class="toc-item-num">3.0.2&nbsp;&nbsp;</span>Number of available variables per row</a></span></li><li><span><a href="#Number-of-available-variables-per-row-for-defined-outcome" data-toc-modified-id="Number-of-available-variables-per-row-for-defined-outcome-3.0.3"><span class="toc-item-num">3.0.3&nbsp;&nbsp;</span>Number of available variables per row for defined outcome</a></span></li><li><span><a href="#Number-of-available-variables-per-row-for-reduced-dataset" data-toc-modified-id="Number-of-available-variables-per-row-for-reduced-dataset-3.0.4"><span class="toc-item-num">3.0.4&nbsp;&nbsp;</span>Number of available variables per row for reduced dataset</a></span></li></ul></li><li><span><a href="#Class-balance" data-toc-modified-id="Class-balance-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Class balance</a></span><ul class="toc-item"><li><span><a href="#Before-removing-row-with-too-many-missing" data-toc-modified-id="Before-removing-row-with-too-many-missing-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Before removing row with too many missing</a></span></li><li><span><a href="#After-removing-row-with-too-many-missing" data-toc-modified-id="After-removing-row-with-too-many-missing-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>After removing row with too many missing</a></span></li></ul></li><li><span><a href="#Features-(left—raw,-right—preprocessed-or-removed)" data-toc-modified-id="Features-(left—raw,-right—preprocessed-or-removed)-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Features (left—raw, right—preprocessed or removed)</a></span></li><li><span><a href="#Outcome-distribution" data-toc-modified-id="Outcome-distribution-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Outcome distribution</a></span></li><li><span><a href="#Missing-data" data-toc-modified-id="Missing-data-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Missing data</a></span></li></ul></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Modeling</a></span><ul class="toc-item"><li><span><a href="#Machine-learning" data-toc-modified-id="Machine-learning-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Machine learning</a></span><ul class="toc-item"><li><span><a href="#Default" data-toc-modified-id="Default-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>Default</a></span></li><li><span><a href="#TODO:-Optimized" data-toc-modified-id="TODO:-Optimized-4.1.2"><span class="toc-item-num">4.1.2&nbsp;&nbsp;</span>TODO: Optimized</a></span></li></ul></li><li><span><a href="#SRTR—Work-in-progress" data-toc-modified-id="SRTR—Work-in-progress-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>SRTR—Work in progress</a></span></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Results</a></span><ul class="toc-item"><li><span><a href="#SRTR" data-toc-modified-id="SRTR-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>SRTR</a></span></li><li><span><a href="#ML" data-toc-modified-id="ML-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>ML</a></span></li></ul></li></ul></div>

In [ ]:
import itertools
import os
from pathlib import Path
import pandas

for folder in itertools.chain([Path.cwd()], Path.cwd().parents):
    if (folder / 'Pipfile').exists():
        os.chdir(folder)
        break

%load_ext autoreload
%load_ext rpy2.ipython
%autoreload 2
%load_ext autotime
%config InlineBackend.figure_format = 'retina'

import numpy as np
from sklearn.compose import ColumnTransformer
from utils import Breakpoint, generate_type_variants
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

from custom_types import Estimator
from notebooks.heart_transplant.heart_transplant_functions import log_transform, filter_out_unused_features, get_X_y_1_year_survival

import pandas

dataset_raw = pandas.read_csv(
    "../cardiovascular-risk-data/data/UNOS.csv",
    dtype={
        'func_stat_tcr': 'str',
    }
)
dataset_raw.columns = [column.lower() for column in dataset_raw.columns]
pandas.set_option('display.max_columns', None)

from functional import pipe
from functools import partial
from heart_transplant_data import heart_transplant_metadata as metadata

In [ ]:
from formatting import format_percents
from heart_transplant_functions import extract_y

n_y_not_defined = extract_y(dataset_raw).isna().value_counts().get(True, 0)

print(f'Outcome not defined: {format_percents(n_y_not_defined/len(dataset_raw))} ({n_y_not_defined})')

# Info

# Dataset

In [ ]:
from functional import pipe
from functools import partial
from heart_transplant_data import heart_transplant_metadata as metadata
from toolz import compose


features = list(metadata.keys())

def feature_engeneering(dataset):
    dataset_new = dataset.copy()
    # TODO: parse
    dataset_new.drop('func_stat_tcr', axis=1, inplace=True)
    return dataset_new

def type_conversion(dataset):
    dataset_new = dataset.copy()
    dataset_new['height ratio'] = pandas.to_numeric(dataset['height ratio'])
    dataset_new['weight ratio'] = pandas.to_numeric(dataset['weight ratio'])
    return dataset_new

def set_na(_X):
    _X_new = _X.copy()
    for column in _X.columns:
        metadata_record = metadata[column]
        try:
            metadata_record['na_values']
        except KeyError:
            pass
        else:
            _X_new[column].replace(metadata_record['na_values'], np.nan, inplace=True)

    return _X_new


dataset_transform_base = compose(
    type_conversion,
    feature_engeneering,
    set_na,
    partial(filter_out_unused_features, metadata=metadata),
)

dataset_without_log = dataset_transform_base(dataset_raw)

dataset = log_transform(dataset_without_log)

X, y = get_X_y_1_year_survival(dataset)

# Preliminary

### Number of available variable per row histogram (raw)

In [ ]:
dataset_raw.apply(lambda x: x.count(), axis=1).hist();

### Number of available variables per row histogram (after preprocessing)

In [ ]:
dataset_without_log.apply(lambda x: x.count(), axis=1).hist();

### Number of available variables per row distribution (removed rows with undefined outcome)

In [ ]:
X.apply(lambda x: x.count(), axis=1).hist();

In [ ]:
missing_mask = X.apply(lambda x: x.count() >= 80, axis=1)


In [ ]:
X_reduced = X[missing_mask]
y_reduced = y[missing_mask]

**Total rows raw**

In [ ]:
print(len(dataset_raw))

**Total rows with defined 1 year survival**

In [ ]:
print(len(X))

**Total rows after removing rows with too many missing variables**

In [ ]:
print(len(X_reduced))

**Percentage of removed rows due to too many missing variables**

In [ ]:
print(format_percents(1-(len(X_reduced)/len(X))))

### Number of available variables per row distributin (removed rows with too many missing variables)

In [ ]:
X_reduced.apply(lambda x: x.count(), axis=1).hist()

## Class balance

### Before removing row with too many missing

In [ ]:
y.value_counts()

### After removing row with too many missing

In [ ]:
y_reduced.value_counts()

In [ ]:
class_counts = y_reduced.value_counts()
class_ratio = class_counts[0]/class_counts[1]


## Features (left—raw, right—preprocessed or removed)

In [ ]:
from formatting import b
from heart_transplant_data import heart_transplant_metadata as metadata
from matplotlib import pyplot


for index, (feature_name, series) in enumerate(dataset_raw.items()):
    metadata_item = metadata.get(feature_name)
    fig, ax = pyplot.subplots(1, 2)
    
    try:
        display_name = f'{metadata_item["name_long"]} ({feature_name})'
    except (KeyError, TypeError):
        display_name = f'{feature_name}'
    
    try:
        series.sample(frac=0.1).hist(grid=False, ax=ax[0])
    except Exception as e:
        print(e)
    
    try:
        X_reduced[feature_name]
    except KeyError:
        ax[1].axis("off")
    else:
        dataset_without_log[feature_name].loc[X_reduced.index].sample(frac=0.1).hist(
            grid=False, ax=ax[1]
        )
    fig.suptitle(f'{index+1}. {display_name}')
    fig.tight_layout()
    fig.show()

## Outcome distribution

In [ ]:
dataset_raw['death'].value_counts(dropna=False)

## Missing data

In [ ]:
import missingno
from matplotlib import pyplot

%matplotlib inline
missingno.matrix(dataset_raw)
pyplot.show()
missingno.matrix(X)
pyplot.show()

In [ ]:
import qgrid

sg = partial(qgrid.show_grid, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200}, show_toolbar=True)

# Modeling

## Machine learning

**TODO**
- cmv_igg PD C (strange values)

In [ ]:
from tempfile import mkdtemp


results_per_method = {}
cache_directory = mkdtemp()

### Default

In [ ]:
from nested_cv import StaticHyperParameters, SimpleEvaluationProtocol
from sklearn.impute import SimpleImputer
from nested_cv import evaluate_simple_cross_validation
from sklearn.preprocessing import OrdinalEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import set_config
from sklearn.linear_model import LogisticRegression
from sklearn_pandas import DataFrameMapper
from utils import use_df
from xgboost import XGBClassifier
from utils import evaluate_and_assign_if_not_present
from sklearn.ensemble import RandomForestClassifier
from toolz import valmap
from nested_cv import get_cv_results_from_simple_cv_evaluation
from functools import partial
from functional import pipe
from sklearn.preprocessing import StandardScaler
from evaluation_functions import compute_classification_metrics_from_results_with_statistics
set_config(display='diagram')

X, y = get_X_y_1_year_survival(dataset)

class Debug(BaseEstimator, TransformerMixin):

    def transform(self, X):
        print(pandas.DataFrame(X).head())
        print(X.shape)
        return X

    def fit(self, X, y=None, **fit_params):
        return self



# for feature in categorical_features:
#     X[feature]=X[feature].astype('category')


categorical_features = [feature_name for feature_name, series in X.items() if (series.dtype == 'object' or feature_name in [
    'iabp_tcr', 'inotropes_tcr', 'ethcat', 'lvad ever', 'gender', 'gender_don', 'tah ever', 'med_cond_trr', 'ecmo_trr', 'gstatus',
    'pstatus', 'ethcat_don', 'blood_inf_don', 'other_inf_don', 'pulm_inf_don', 'urine_inf_don', 'cod_cad_don',
    'death_mech_don', 'multiorg', 'abo_mat', 'lv_eject_meth', 'coronary_angio', 'vessels_50sten', 'biopsy_dgn',
    'ecd_donor', 'education', 'education', 'congenital', 'prior_card_surg_type_tcr', 'ventilator_tcr', 'rvad ever', 'cancer_site_don',
])]

continuous_features = [
    feature_name for feature_name in X.columns if feature_name not in categorical_features
]


keys = [ X[column].unique() for column in categorical_features ]
feature_names = [*categorical_features, *continuous_features]

transform_pipeline = make_pipeline(
    use_df(ColumnTransformer, columns=feature_names)([
         (
             'categorical',
             make_pipeline(
                 SimpleImputer(strategy='most_frequent'),
                 OrdinalEncoder(categories=keys)
             ),
             categorical_features,
         ),
         ('continuous', SimpleImputer(strategy='mean'), continuous_features),
     ]),
)


evaluate_and_assign_if_not_present(
    results_per_method,
    'xgboost_balanced',
    lambda: evaluate_simple_cross_validation(
        lambda: make_pipeline(
            transform_pipeline,
            StandardScaler(),
            use_df(XGBClassifier)(
            eval_metric='logloss',
                scale_pos_weight=class_ratio,
                use_label_encoder=False,
                n_jobs=12,
            )
        ),
        X_reduced,
        y_reduced,
        StaticHyperParameters({}),
        SimpleEvaluationProtocol(repeats=1, cv=10),
        parallel=False,
        feature_names=feature_names,
    )
)

evaluate_and_assign_if_not_present(
    results_per_method,
    'xgboost',
    lambda: evaluate_simple_cross_validation(
        lambda: make_pipeline(
            transform_pipeline,
            StandardScaler(),
            use_df(XGBClassifier)(
            eval_metric='logloss',
                use_label_encoder=False,
                n_jobs=12,
            )
        ),
        X_reduced,
        y_reduced,
        StaticHyperParameters({}),
        SimpleEvaluationProtocol(repeats=1, cv=10),
        parallel=False,
        feature_names=feature_names,
    )
)

evaluate_and_assign_if_not_present(
    results_per_method,
    'random_forest',
    lambda: evaluate_simple_cross_validation(
        lambda: make_pipeline(
            transform_pipeline,
            StandardScaler(),
            use_df(RandomForestClassifier)()
        ),
        X_reduced,
        y_reduced,
        StaticHyperParameters({}),
        SimpleEvaluationProtocol(repeats=1, cv=10),
        parallel=False,
        feature_names=feature_names,
    )
)

evaluate_and_assign_if_not_present(
    results_per_method,
    'logistic_regression',
    lambda: evaluate_simple_cross_validation(
        lambda: make_pipeline(
            transform_pipeline,
            StandardScaler(),
            LogisticRegression(),
        ),
        X_reduced,
        y_reduced,
        StaticHyperParameters({}),
        SimpleEvaluationProtocol(repeats=1, cv=10),
        parallel=False,
        feature_names=feature_names,
    )
)


metrics_per_method = pipe(
    results_per_method,
    partial(valmap, get_cv_results_from_simple_cv_evaluation),
    partial(valmap, partial(compute_classification_metrics_from_results_with_statistics, y))
)


### TODO: Optimized

In [ ]:

evaluate_and_assign_if_not_present(
    results_per_method,
    'xgboost_balanced_optimized',
    lambda: evaluate_simple_cross_validation(
        lambda: make_pipeline(
            transform_pipeline,
            StandardScaler(),
            use_df(XGBClassifier)(
            eval_metric='logloss',
                scale_pos_weight=class_ratio,
                use_label_encoder=False,
                n_jobs=12,
            )
        ),
        X_reduced,
        y_reduced,
        BayesianOptimization(XGBoostMethod.get_hyperopt_space()),
        SimpleEvaluationProtocol(repeats=1, cv=10),
        parallel=False,
        feature_names=feature_names,
    )
)

## SRTR—Work in progress

In [ ]:
from heart_transplant_functions import reverse_log_transform_row
from pandas import Series


def predict_srtr(row: Series) -> float:
    row_raw = reverse_log_transform_row(row)
    outcome = 0.
    
    if row['diab'] == 1:
        outcome += -0.066605837
    elif pandas.isna(row['diab']):
        outcome += -0.066605837
        
    if row['education'] == 4:
        outcome += -0.000272643
    elif pandas.isna(row['education']):
        outcome += -0.000272643
        
    if row['cig_use'] == 'Y':
        outcome += 0.065904842
    
    if row['malig'] == 'Y':
        outcome += 0.02219013
        
    if row['ethcat'] == 2:
        outcome += -0.088669408
    elif row['ethcat'] == 7:
        outcome += -0.434641386
    elif row['ethcat'] == 1:
        outcome += 0.016172921
    elif pandas.isna(row['ethcat']):
        outcome += -0.434641386
    
    if row['hist_oth_drug_don'] == 'N':
        outcome += 0.003011283
        
    if row['hist_cig_don'] == 'Y':
        outcome += 0.000498645
    
    if row_raw['diag'] == 1201 or row_raw['diag'] == 1208:
        outcome += -0.11598044
    elif pandas.isna(row_raw['diag']):
        outcome += -0.11598044
    
    if row_raw['age_don'] > 55:
        outcome += 0.022276125
    elif row_raw['age_don'] > 50:
        outcome += 0.024025803
    elif row_raw['age_don'] > 40:
        outcome += 0.024025803
    elif row_raw['age_don'] > 20:
        outcome += 0.001105558
    elif row_raw['age_don'] > 15:
        outcome += 0.0000457    
        
        
    if row['hist_hypertens_don'] == 'Y':
        outcome += 0.000140302
        
    if row_raw['bmi_don_calc'] < 22:
        outcome += -0.013675144
    elif pandas.isna(row_raw['bmi_don_calc']):
        outcome += -0.054750638
    
    
    if row_raw['bun_don'] < 10:
        outcome += 0.006214231
    elif row_raw['bun_don'] < 15:
        outcome += 0.030163193
    elif row_raw['bun_don'] < 25:
        outcome += 0.000675563
        
    #TODO: We have values 1-4, mapped to the parameters from excel file
    # Cerebrovascular/Stroke
    if row['cod_cad_don'] == 2:
        outcome += 0.009943402
    
    if row['gender_don'] == 'F':
        outcome += 0.131377454
    
    if row['donor insulin'] == 'Y':
        outcome += -0.243727342
    elif pandas.isna(row['donor insulin']):
        outcome += -0.243727342
    
    if row['ethcat_don'] == 9:
        outcome += -0.005654728
    elif row['ethcat_don'] == 7:
        outcome += -0.022069858
    elif row['ethcat_don'] == 1:
        outcome += 0.007700082
    elif pandas.isna(row['ethcat_don']):
        outcome += -0.022069858
    
    if row_raw['height ratio'] > 1.08:
        outcome += 0.123483738
    elif row_raw['height ratio'] > 1.06:
        outcome += 0.177597673     
    elif row_raw['height ratio'] > 0.98:
        outcome += 0.177597673
        
    if row_raw['weight ratio'] < 0.7:
        outcome += -0.676215593
    elif row_raw['weight ratio'] > 1.6:
        outcome += -0.616706456
    elif row_raw['weight ratio'] > 1.3:
        outcome += -0.333586524
    elif pandas.isna(row_raw['weight ratio']):
        outcome += -0.32244479
    
    if row_raw['creat_don'] < 0.8:
        outcome += -0.0000895
    elif pandas.isna(row_raw['creat_don']): 
        outcome += -0.003848936

    if row_raw['age'] < 70:
        outcome += -0.0048657
    elif row_raw['age'] > 40:
        outcome += 0.011148729
    elif pandas.isna(row_raw['age']):
        outcome += -0.240346908
    
    if row_raw['bmi_calc'] > 36:
        outcome += 0.156816331
    elif row_raw['bmi_calc'] > 34:
        outcome += 0.015018691
    elif row_raw['bmi_calc'] > 30:
        outcome += 0.002490638
    
    if row_raw['hemo_co_tcr'] > 3.5:
        outcome += -0.000561188
    elif pandas.isna(row_raw['hemo_co_tcr']):
        outcome += -0.002749823
    
    if row_raw['most_rcnt_creat'] < 0.7:
        outcome += 1.751207911
    elif row_raw['most_rcnt_creat'] > 1.5:
        outcome += 0.161043865
    elif row_raw['most_rcnt_creat'] > 1:
        outcome += 0.355230656
    elif row_raw['most_rcnt_creat'] > 0.9:
        outcome += 0.004575029
    
    if row_raw['dial_after_list'] == 'Y':
        outcome += 0.484377854
    
    if row_raw['ecmo_trr'] == 1:
        outcome += 0.350887035
    
    # TODO ischemic time, correct variable?
    if row_raw['ischtime'] < 120:
        outcome += -0.00088359
    elif row_raw['ischtime'] < 140:
        outcome += -0.000286891
    elif row_raw['ischtime'] < 160:
        outcome += -0.0000401  
    elif row_raw['ischtime'] < 300:
        outcome += 0.000587559
    
    if pandas.isna(row_raw['ischtime']):
        outcome += -0.218530997
        
    if row_raw['med_cond_trr'] != 'ICU':
        outcome += -0.075556547
        
    if row_raw['hemo_pcw_tcr'] < 16:
        outcome += 0.001151401
    elif row_raw['hemo_pcw_tcr'] < 18:
        outcome += 0.000863272
    elif row_raw['hemo_pcw_tcr'] > 32:
        outcome += 0.000288132
    elif row_raw['hemo_pcw_tcr'] > 30:
        outcome += 0.016847899
    elif row_raw['hemo_pcw_tcr'] > 28:
        outcome += 0.013182796
    
    if row_raw['retransplant'] >= 1:
        outcome += 0.105895416
        
    if row_raw['hemo_pa_dia_tcr'] > 32:
        outcome += 0.003828561
    elif row_raw['hemo_pa_dia_tcr'] > 30:
        outcome += 0.00102561
        
    if row_raw['hemo_sys_tcr'] < 20:
        outcome += 0.002339633
    elif row_raw['hemo_pa_dia_tcr'] < 25:
        outcome += 0.003705285
    elif row_raw['hemo_pa_dia_tcr'] > 65:
        outcome += 0.002035943
    
    if row_raw['rvad ever'] == 1:
        outcome += 0.025412411

    if row_raw['tah ever'] == 1:
        outcome += 0.740901872

    if row_raw['tbili'] > 2:
        outcome += 0.29652491
    elif row_raw['tbili'] > 0.4:
        outcome += 0.199733923
    
    if row_raw['transfusions'] == 'Y':
        outcome += 0.289314026
    
    if row_raw['ventilator_tcr'] == 1:
        outcome += 0.289314026
    
    return outcome

In [ ]:
y_pred_srtr = X.apply(predict_srtr, axis=1)

In [ ]:
from utils import inverse_binary_y

d = pandas.DataFrame({
    'y': inverse_binary_y(y),
    'y_pred_srtr': y_pred_srtr
})


# Results

## SRTR

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(inverse_binary_y(y), y_pred_srtr)

## ML

In [ ]:
from visualisation import list_of_lists_to_html_table, display_html
from formatting import compare_metrics_in_table

display_html(list_of_lists_to_html_table(compare_metrics_in_table(metrics_per_method)))